# Build Annotator Tutorial

The current notebook will demonstrate how to build new annotators. 

Ipyannotator architecture uses four main layers:
- The **View** is the layer responsable for render the visualization. Ipyannotator uses [ipycanvas](https://ipycanvas.readthedocs.io/en/latest/) and [ipywidgets](https://ipywidgets.readthedocs.io/en/latest/) to structure and mount the visualization layer, but we also develop some internal components like the Navi (that help our users to navigate throught the images that will be annotated).
- The **Storage** is the layer that receive the data and stores. Ipyannotator uses different types of storage like txt, sqlite.
- The **Controller** is the layer that acts like a mediator between the state, storage and view. This layers tells when the information from the state will be stored. 
- **"Model/State (in memory)"** is the central place of the Ipyannotator layer structure is the "Model/State (in memory)" that centralize the data and make sure to sync it across the application. If something changes on the Model/State then other layers can listen and sync the information.


The image below shows an example of how the layers are structured and how its communications are made. 

The annotator developed in the current notebook was a minimal one called CircleAnnotator, that draws a circle every time a user clicks on the canvas.

![New Component Diagram for annotators](https://www.plantuml.com/plantuml/png/VPA_ReCm48TtFuNbgLt82o0KIXUaP2b3vsjmAPB_87EqYYhUlHUZOEEsP0du--xym-VZYE1mqegrWF06e-JYR5kf3Wq2IlxG6wwbjkxwA3YCl3Pd_yQ_6PkPanS4qoagAeVXjMyxYLvRtyZZz0iYaTeCqntmBJt9TooTir-t9g8en5_IIzzzH5QUBzQSx2GgC9ymHgdYXR3_xn8VCAf80YT5dfPxy6aFLdlmUONp_t6Zf4b8kA1rD9lRdxVeAUauqnZNeUQPoehDCIKi33QQVMKDEgkKT2myaZ-HVo-Fz8R2G2TFSDqM-FuRkDwLp14AQhYhfJ4M0NjhwDCvcmCH3UX1oV5GQ-gtdD6ov1T89plUZGAtMH5tPB5FU4hp7QLf9wr-0000)

## Model/State (in memory)

To develop a model/state ipyannotator uses [Pydantic models](https://pydantic-docs.helpmanual.io/usage/models/) to assert the data type of the output model. Every change made in a state is triggered using [PyPubSub](https://pypubsub.readthedocs.io/en/v4.0.3/) and this events can be listen to other layers to assert the sync between components.

For the `CircleAnnotator` we'll split the data into two states:

- **AppWidgetState** it's a common state for all annotators. The `AppWidgetState` stores the canvas size, navi index and max number of images. You can use him to communicate with the Ipyannotator navigation component (Navi) or on your own custom navigation component.
- **CircleAnnotatorState** it's the state responsable to store the `CircleAnnotator` data, it will store the circle radius, view layers, current image and circle drawn.

**Observation:** The model/state doesn't have to be restricted to a single class (as shown in the image above), its data should make sense accordlying to the structure of the annotator. 

In [ ]:
from pubsub import pub
from typing import Tuple, List, Dict, Optional
from ipyannotator.base import BaseState, AppWidgetState
from abc import ABC, abstractmethod
from IPython.core.display import display

In [ ]:
class CircleAnnotatorState(BaseState):
    radius: float = 30
    current_frame: Optional[str]
    circles: Dict = {}
    layers: Dict[str, int] = {
        'bg': 0,
        'image': 1,
        'circle': 2,
    }

## View

The view layer should store all ipywidgets that will be used on the annotator. The next commands will start the GUI development of the CircleAnnotator.

In [ ]:
from ipywidgets import AppLayout, VBox, HBox, Layout, Output, Image
from ipycanvas import MultiCanvas
from pathlib import Path
from ipyannotator.navi_widget import Navi
from ipyannotator.bbox_canvas import draw_img, draw_bg
from ipyannotator.debug_utils import IpyLogger
from ipyannotator.storage import MapeableStorage, get_image_list_from_folder

The `CircleCanvas` class will be a component of our GUI, allowing to draw circles, backgrounds, images and also clear them.

In [ ]:
class CircleCanvas(HBox):
    debug_output = Output(layout={'border': '1px solid black'})

    def __init__(self, width: float, height: float, layers: dict):
        super().__init__()

        self._multi_canvas = MultiCanvas(
            len(layers),
            width=width,
            height=height,
        )

        children = [VBox([self._multi_canvas])]
        self.children = children

    def clear(self, layer: int):
        self._multi_canvas[layer].clear()

    def draw_circle(self, layer: int, x: float, y: float, radius: float):
        self._multi_canvas[layer].stroke_circle(x, y, radius)

    def _draw_bg(self, layer: int = 0):
        draw_bg(self._multi_canvas[layer])

    def draw_image(self, layer: int, image_path: str):
        draw_img(self._multi_canvas[layer], Path(image_path), clear=True)

In [ ]:
circle_canvas = CircleCanvas(width=200, height=200, layers={'image': 1, 'bg': 0, 'circle': 2})
circle_canvas.draw_image(0, '../data/projects/bbox/pics/blueSquare800x600.png')

In [ ]:
circle_canvas.debug_output

In [ ]:
circle_canvas

In [ ]:
circle_canvas.draw_circle(1, 63, 62, 15)

The ` CircleAnnotatorGUI ` is our view. This layer communicates with the states, for example, if the state index changes our view will clear the draw layer, change the image and redraw the circles that were load to the state.

In [ ]:
class CircleAnnotatorGUI(AppLayout):
    debug_output = Output(layout={'border': '1px solid black'})

    def __init__(self, app_widget: AppWidgetState, circle_state: CircleAnnotatorState):
        self._app_widget = app_widget
        self._circle_state = circle_state

        self._navi = Navi()

        self._controls_box = HBox(
            [self._navi],
            layout=Layout(
                display='flex',
                flex_flow='row wrap',
                align_items='center'
            )
        )

        self._image_box = CircleCanvas(
            width=self._app_widget.size[0],
            height=self._app_widget.size[1],
            layers=self._circle_state.layers
        )

        self._listen_index_changes()
        self._listen_click()
        self._listen_max_im_number_changes()
        self._navi.on_navi_clicked = self._update_index

        super().__init__(
            header=None,
            left_sidebar=None,
            center=self._image_box,
            right_sidebar=None,
            footer=self._controls_box,
            pane_widths=(2, 8, 0),
            pane_heights=(1, 4, 1))

    def _listen_click(self):
        layer = self._circle_state.layers['circle']
        self._image_box._multi_canvas[layer].on_mouse_down(self._draw_circle)

    def _draw_circle(self, x: float, y: float, radius: float = None, append=True):
        layer = self._circle_state.layers['circle']

        draw = {
            'x': x,
            'y': y,
            'radius': radius or self._circle_state.radius,
        }

        if append:
            self._circle_state.circles[self._circle_state.current_frame].append(draw)

        self._image_box.draw_circle(layer, draw['x'], draw['y'], draw['radius'])

    def _draw_circle_from_state(self, frame: str):
        circles = self._circle_state.circles[frame]

        for circle in circles:
            circle['append'] = False
            self._draw_circle(**circle)

    def _draw_image(self, image_path: str):
        image_layer = self._circle_state.layers['image']
        self._image_box.draw_image(image_layer, image_path)

    def _listen_max_im_number_changes(self):
        self._update_max_navi_slider(self._app_widget.max_im_number)
        self._app_widget.subscribe(self._update_max_navi_slider, 'max_im_number')

    def _update_max_navi_slider(self, max_im_number: int):
        self._navi.max_im_num = max_im_number

    def _listen_index_changes(self):
        if self._circle_state.current_frame:
            self._change_image(self._circle_state.current_frame)
        self._circle_state.subscribe(self._change_image, 'current_frame')

    def _change_image(self, current_frame: str):
        self._image_box.clear(self._circle_state.layers['circle'])
        self._draw_image(current_frame)
        self._draw_circle_from_state(current_frame)

    def _update_index(self, index: int):
        self._app_widget.index = index

## Storage

Ipyannotator uses json as a data structure to store the annotation data. The package also allows the users to change the type of storage accordlying to the users needs, for example, you can store your data in files or databases like sqlite. In this example it's developed a `Storage` module that will keep our data in memory (using the `InMemoryStorage` class).

In [ ]:
class Storage(ABC):
    @abstractmethod
    def bulk_annotation(self, index: int, annotation: list):
        pass

    @abstractmethod
    def find(self, index: int):
        pass

In [ ]:
class InMemoryStorage(Storage, MapeableStorage):
    def __init__(
        self,
        image_dir: Path,
    ):
        super().__init__()
        self.images = get_image_list_from_folder(image_dir)
        self.update({str(image): [] for image in self.images})

    def get_image(self, index: int) -> str:
        return str(self.images[index])

    def bulk_annotation(self, index: int, annotations: list):
        image_path = self.get_image(index)
        self.mapping[image_path] = annotations

    def find(self, index: int):
        image_path = self.get_image(index)
        return self.__getitem__(image_path)

## Controller

The controller serves as a mediator between the states, the gui and the storage. This layer listens for states changes and stores the data on the storage, it also can load the storage data into the states.

To demonstrate how the communication works, the `IpyLogger` class can be used as a decorator to output all the pubsub communication into the logger. The `pub.ALL_TOPICS` parameter will get all the messages.

In [ ]:
logger = IpyLogger('CircleLogger')

In [ ]:
logger.show_logs()

In [ ]:
@logger.subscribe('AppWidgetState')
class CircleAnnotatorController:
    def __init__(
        self,
        app_widget: AppWidgetState,
        circle_state: CircleAnnotatorState,
        storage: Storage
    ):
        self._last_index = 0
        self._app_widget = app_widget
        self._circle_state = circle_state
        self._storage = storage
        self._storage_to_state()

        app_widget.subscribe(self._update_current_frame, 'index')

    def _storage_to_state(self):
        logger.info("-> _storage_to_state")
        self._circle_state.circles = self._storage.mapping
        self._update_current_frame()
        self._update_max_im_number()

    def _update_max_im_number(self):
        logger.info("-> _update_max_im_number")
        self._app_widget.max_im_number = len(self._storage)

    def _update_current_frame(self, index: int = 0):
        self._save_annotation(self._last_index)
        # error: "Storage" has no attribute "get_image"
        image_path = self._storage.get_image(index)  # type: ignore
        self._circle_state.current_frame = image_path
        self._last_index = index

    def _save_annotation(self, index: int):
        logger.info("-> _save_annotation")
        # error: "Storage" has no attribute "get_image"
        image_path = self._storage.get_image(index)  # type: ignore
        annotations = self._circle_state.circles[image_path]
        self._storage.bulk_annotation(index, annotations)

In [ ]:
storage = InMemoryStorage(Path('../data/projects/bbox/pics'))

In [ ]:
app_widget = AppWidgetState()

In [ ]:
circle_state = CircleAnnotatorState()

In [ ]:
controller = CircleAnnotatorController(app_widget, circle_state, storage)

In [ ]:
CircleAnnotatorGUI(app_widget, circle_state)

In [ ]:
logger.show_logs()

## Annotator

Ipyannotator's design can be described by three properties: input, output, actions. The goal is to develop flexible modules with a common interface.

With all `CircleAnnotator` layers developed we can now create it's single instance. For the current annotator the properties used are:

- input: Image
- output: Circle
- actions: explore, improve, create

In [ ]:
from ipyannotator.mltypes import InputImage, Input, Output

In [ ]:
class CircleOutput(Output):
    pass

In [ ]:
class CircleAnnotator:
    def __init__(
        self,
        project_path: Path,
        input_item: InputImage,
        output_item: Output,
        *args, **kwargs
    ):
        self._app_widget = AppWidgetState(uuid=str(id(self)), **{
            'size': (input_item.width, input_item.height)
        })
        self._circle_state = CircleAnnotatorState(uuid=str(id(self)))

        self._storage = InMemoryStorage(project_path / input_item.dir)

        self._controller = CircleAnnotatorController(
            self._app_widget,
            self._circle_state,
            self._storage
        )

        self._view = CircleAnnotatorGUI(
            self._app_widget,
            self._circle_state
        )

    def __repr__(self):
        display(self._view)
        return ""

In [ ]:
in_p = InputImage(image_dir='pics', image_width=600, image_height=400)
out_p = CircleOutput()

circle_annotator = CircleAnnotator(
    project_path=Path('../data/projects/bbox/'),
    input_item=in_p,
    output_item=out_p
)

circle_annotator

## Diagram

The following sequence diagram shows how the CircleAnnotator comunicates with its components when a user clicks on the next button navigation.

![sequence diagram](https://plantuml.palaimon.io/png/XLFDRjim3BxhARZcqXtw0aOH84w0Oi0s13227Oh2e6NM5QB8dYIdoPv-bAMaTcfi9rlKzqFoizrUcGuj7i3HxvwCf1_a73QqqgenO5NpviMNpc9pGF3K_W5lUn9Y9NrhOUV82b6r9w2JBpnwWaMdp5wmf5TITMWyhBhkbweRIW1qW5qNts42N2ihDQsCQVcojGD4aAc13QBBtTFksnqileUkpgHr-yxtNldpBPTnWn6Vdtfr0Vt4eqet9ZmNEWXLkYUOwgpH7D4bg1oNUfLOZIKo7zt9rdZRwi5YdTw3pTfRFVAvucxwJHHDTkJuGOrcWWFYLhUxlDZb0RSjGDJeiK97KB0CvnRgcro840qyBEEW6HWEgF963CTG3kePX-vBPMewYTWftrp2oQ3lM9pYFJnArBf2kL_2SsbuoZ8KiDBq8b1wTHooJLnnJPW5jqK6ZpodUZqpLzMd5r7J3ELIsHQ21tjOBaTxoA1CtSZUMiwgVELlbiP2J1EZnR7n9i-WwlM-nBXcrHfthz6baR_Em9kmZmElyVxYaw1N6zxj9W_mSNEMV6yD34ptFA5EXcNor7Fkau-fJEUyAip-8tF5pvkVA4xQcXMToIFzJXo6V4FJK5pLRGW9DThHvTV9W4ze8Stq3rnjyJsf_nSMg-ul)